In [35]:
import findspark
findspark.init()
import pandas as pd;
import csv;
import numpy as np;
from pyspark import SparkContext
from pyspark import SparkConf
import re

In [36]:
conf = SparkConf().setAppName('Read & parse text data file in pyspark')
sc = SparkContext.getOrCreate(conf=conf)

In [37]:
lines = sc.textFile('./assets/AMiner-Paper.txt').zipWithIndex()

In [39]:
# Find the starting row of each data entry
pos = lines.filter(lambda x: "#index" in x[0]).map(lambda x: x[1]).collect()

In [40]:
zeroArray = np.zeros(lines.count(), dtype=int)

In [41]:
def exchangeElementAtIndexToOne(index):
    zeroArray[index] = 1;

In [42]:
for element in pos:
    exchangeElementAtIndexToOne(element)

In [43]:
# Calculate cumulative sum of starting data entry indexes
summedArray = np.cumsum(zeroArray)

In [44]:
broadcastedArray = sc.broadcast(summedArray);

In [45]:
def createBroarcastedTuple(x):
    return (broadcastedArray.value[x[1]], x[0])

In [46]:
papers = lines.map(lambda x: createBroarcastedTuple(x))

In [47]:
list_get = lambda l, x, d=None: d if not l[x] else l[x] # safe getter of list values

paper_keys = ["#index", "#*", "#@", "#o", "#t", "#c", "#%", "#!"]

# author_keys = ["#index", "#n", "#a", "#pc", "#cn", "#hi", "#pi", "#upi", "#t"]
# AUTHOR_KEYS_MAP = {
#     "#index": "id", "#n": "name", "#a": "affiliations", "#pc": "paper_count", "#cn": "citation_count", "#t": "research_interests", "#hi": "h_index", "#upi": "u_p_index", "#pi": "p_index"
# }

PAPER_KEYS_MAP = {
    "#index": "id", "#*": "title", "#@": "authors", "#o": "affiliations", "#t": "year", "#c": "publication_venue", "#%": "ref_ids", "#!": "abstract",
}

with open('./assets/parsedData/authors.csv', 'w', encoding='utf8', newline='') as output_file:
    fc = csv.DictWriter(output_file, fieldnames=paper_keys)
    fc.writeheader()
#     fc.writerows(mappedPaperDicts.collect())

def splitIntoKeyValue(stringToSplit):
    formattedValue = stringToSplit 
    if type(stringToSplit) is str:
        if len(formattedValue) > 0:
            formattedValue = formattedValue.split(" ", 1)
            key = list_get(formattedValue, 0)
            mappedKey = PAPER_KEYS_MAP.get(key) # note : this is not ideal. We should check and write better
#             print(">>> ", mappedKey)
            if (mappedKey):
                formattedValue = { mappedKey: list_get(formattedValue, 1, '') }
            else:
                print("-> ", key, stringToSplit)
                formattedValue = {}
        else:
            formattedValue = {}
    return formattedValue

def reduceFeaturesToDict(featureA='', featureB=''):
#     print("here")
    try:        
        splittedA = splitIntoKeyValue(featureA);
        splittedB = splitIntoKeyValue(featureB);
#         print("->>>>-->",splittedA)
#         if type(splittedA) is str:
#             if len(splittedA) > 0:
#                 splittedA = splittedA.split(" ", 1)
#                 mappedKey = AUTHOR_KEYS_MAP[list_get(splittedA, 0)]
#                 if (mappedKey):
#                     splittedA = { mappedKey: list_get(splittedA, 1, '') }
#                 else:
#                     splittedA = {}
#             else:
#                 splittedA = {}

#         splittedB = featureB;
#         if type(featureB) is str:
#             if len(featureB) > 0:
#                 splittedB = featureB.split(" ", 1)
#                 splittedB = { list_get(splittedB, 0): list_get(splittedB, 1, '') }
#             else:
#                 splittedB = {}
#         if 
        return { **splittedA, **splittedB }
    except:
        print("ERROR: ", featureA, featureB)
        raise

In [48]:
reducedPapers = papers.reduceByKey(lambda a, b: reduceFeaturesToDict(a, b))

In [49]:
print(reducedPapers.sortByKey(ascending=False).first())

/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apache-spark/3.2.0/libexec/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/usr/local/Cellar/apac

(2092356, {'id': '2092356', 'title': 'Reliability prediction through system modeling', 'authors': 'Lalit Kumar Singh;Gopika Vinod;A. K. Tripathi', 'affiliations': 'Dept of Computer Engg IIT(BHU) Varanasi, India;Reactor Safety Division Bhabha Atomic Research Centre Dept of Atomic Energy, Govt of India;Dept of Computer Engg IIT(BHU) Varanasi, India', 'year': '2013', 'publication_venue': 'ACM SIGSOFT Software Engineering Notes', 'ref_ids': '1056157', 'abstract': 'Quantifying software reliability, such as performance and dependability, through stochastic behavior models (or labeled transition systems) is already a common practice in the software analysis community. However, those models are usually too fine grained to represent an accurate view of the software system by its stakeholders. Scenarios, on the other hand, are capable not only to describe the system traces as behavior models do but also depict very clearly the system components designed to provide the intended system behavior as

In [51]:
mappedPaperDicts = reducedPapers.map(lambda x: x[1]) # retrieve dicts from the tuples

In [52]:
df = sqlContext.createDataFrame(mappedPaperDicts)

In [53]:
# Save data to csv file
df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "true").save("papers.csv")

In [ ]:
mappedPaperDicts.saveAsTextFile('./assets/parsedData/authors')